## chap04

1.   データが置かれたGoogleドライブをマウント（Colabのメニューで操作）


> ファイル -> ドライブをマウント -> 実行可能なセルが現れる -> ログインして認証コードを入力




2.   /content/driveにマウントされる



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


GiNZAのインストール  

In [3]:
!pip install -U ginza
import pkg_resources, imp
imp.reload(pkg_resources)

     |████████████████████████████████| 54.9MB 78kB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 44.8MB 88kB/s 
     |████████████████████████████████| 481kB 45.0MB/s 
  Created wheel for ginza: filename=ginza-3.1.2-cp36-none-any.whl size=17311 sha256=98e689ba3e4aace681f5f5d6a8811979ef38a2f8eb9e6c4a196629b12b13c411
  Stored in directory: /root/.cache/pip/wheels/70/8d/57/f089078acc0dbaebffc08c178e9f20924fa794c114ad36f7f7
  Created wheel for ja-ginza: filename=ja_ginza-3.1.0-cp36-none-any.whl size=54963619 sha256=e07c2639b3516d717b9a7226e9b4daa80c1b70e752ff20405b4a351760b2974a
  Stored in directory: /root/.cache/pip/wheels/f2/8a/07/1837eeb5c5648fa8d266102b78a894e495234585ac3f024cf1
  Created wheel for ja-ginza-dict: filename=ja_ginza_dict-3.1.0-cp36-none-any.whl size=70877544 sha256=9253e3d0c85e8590257f7c22f0957d230a992c0c1407e2a30d680539d9205b79
  Stored in directory: /root/.cache/pip/wheels/30/88/d7/7f0692ba26060966af34538e1079438d166

<module 'pkg_resources' from '/usr/local/lib/python3.6/dist-packages/pkg_resources/__init__.py'>

GiNZAの使用準備

In [0]:
import spacy
nlp = spacy.load('ja_ginza')

京都大学情報学研究科--NTTコミュニケーション科学基礎研究所 共同研究ユニット 
 [解析済みブログコーパス](http://nlp.ist.i.kyoto-u.ac.jp/kuntt/#ga739fe2)
の読み込み

In [0]:
Dpath = '/content/drive/My Drive/MMI_exercise/corpus2/'
topics = ['Gourmet', 'Keitai', 'Kyoto', 'Sports']

In [0]:
import pandas as pd
import re

cols = ['doc', 'label']
df = pd.DataFrame(index=[], columns=cols)
p = re.compile(r'(［.*］)(.*)')

for topic in topics:
  data = pd.read_table(Dpath+topic+'.tsv', header=None, dtype=str, usecols=[1])
  for d in data.itertuples():
    m = p.match(d[1])
    if m != None:  # first line of the document
      if d[0] != 0:
        df = df.append(pd.DataFrame([doc.strip(), topic], index = cols).T)
      sent = m.groups()[1]
      doc = ''
    else:
      sent = d[1]
    doc = doc + ' ' +  ' '.join([t.text for t in nlp(sent)])
  df = df.append(pd.DataFrame([doc.strip(), topic], index = cols).T)

In [7]:
df

,doc,label
0,烏丸 六角 の おかき 屋 さん 六角堂 の 前 に ある 、 蕪村 庵 と いう お店 に...,Gourmet
0,河原町 の 居酒屋 この 間 先輩 たち に つれ られ 、 河原町 の 居酒屋 へ 行っ ...,Gourmet
0,ちょっと 贅沢 ほんの ちょこっと な ん だ けど 、 贅沢 し たい とき に 何 を ...,Gourmet
0,食欲 の 秋 最近 おなか が 減っ て 仕方 あり ませ ん か 。 仕方 あり ませ ん...,Gourmet
0,季節 限定 も のっ て … １０ 月 と いえ ば 秋 本番 で ある 。 栗 や 秋刀魚...,Gourmet
...,...,...
0,クライマックス シリーズ 今年 の 阪神 は なん やっ た ん や ！ ！ 初め 弱く て...,Sports
0,京都 サンガ に つい て この ブログ の 右側 に 検索 窓 ある じゃ ない です か...,Sports
0,サッカー から 伺える ルール と の 付き合い 方 私自身 は 全く の サッカー 素人 ...,Sports
0,する か 見る か スポーツ は する 派 です か ？ し たい 見る 派 です か ？ ...,Sports


In [8]:
df['label'].value_counts(sort=False)

Sports     22
Kyoto      91
Gourmet    57
Keitai     79
Name: label, dtype: int64

# 文書分類

## 単語の出現頻度を数える

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
bow = vectorizer.fit_transform(df['doc'])
bow = bow.toarray()
print(vectorizer.get_feature_names())

print(bow)

# vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
# X2 = vectorizer2.fit_transform(df['doc'])
# print(vectorizer2.get_feature_names())

['〇〇', 'ぁん', 'あぁ', 'ああ', 'あい', 'あいつ', 'あう', 'あえて', 'あか', 'あがら', 'あき', 'あきれ顔', 'あくどい', 'あくまで', 'あけ', 'あげ', 'あげる', 'あげれ', 'あたかも', 'あたし', 'あたっ', 'あたり', 'あたりまえ', 'あちこち', 'あちら', 'あっ', 'あっさり', 'あっさり味', 'あったかい', 'あったかく', 'あつい', 'あて', 'あてれ', 'あと', 'あなた', 'あの', 'あふれ', 'あふれかえっ', 'あふれる', 'あぶら', 'あまり', 'あら', 'あらかじめ', 'あらゆる', 'あらわし', 'あり', 'ありがた', 'ありがたい', 'ありがたみ', 'ありがとう', 'ありきたり', 'ありふれ', 'ありゃ', 'ある', 'あるいは', 'あれ', 'あれこれ', 'あろう', 'あわせ', 'あわせる', 'あん', 'あんかけ', 'あんな', 'あんまり', 'あんみつ', 'あー', 'いい', 'いいわけ', 'いぅ', 'いう', 'いえ', 'いえる', 'いか', 'いかが', 'いかに', 'いき', 'いきなり', 'いく', 'いくつ', 'いくら', 'いけ', 'いける', 'いこ', 'いこう', 'いざ', 'いじっ', 'いじめ', 'いじり', 'いずれ', 'いそい', 'いたく', 'いたし', 'いただき', 'いただけ', 'いただける', 'いたって', 'いたる', 'いち', 'いちいち', 'いちよう', 'いっ', 'いっそう', 'いったい', 'いったん', 'いっぱい', 'いつ', 'いつも', 'いとこ', 'いとも', 'いま', 'いまいち', 'いまさら', 'いまだ', 'いも', 'いや', 'いやいや', 'いやみ', 'いやー', 'いよいよ', 'いよう', 'いら', 'いらっ', 'いらっしゃる', 'いり', 'いる', 'いれ', 'いろいろ', 'いろんな', 'いわ', 'いわく', 'いわば', 'いわゆる', 'いーっぱい', 'うえ', 'うかがえ', 'うがっ', 'うきうき', 'うじう', 'うずら', 'うそ

### ナイーブベイズ

In [16]:
# import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

X = bow
y = df['label']

clf = MultinomialNB()

scores=cross_val_score(clf, X, y, cv=10)
print("acc = ", scores.mean())
print("std = ", scores.std())

acc =  0.8996666666666666
std =  0.06251133230596406


### ロジスティック回帰

In [17]:
from sklearn.linear_model import LogisticRegression

X = bow
y = df['label']

clf = LogisticRegression()

scores=cross_val_score(clf, X, y, cv=10)
print("acc = ", scores.mean())
print("std = ", scores.std())

acc =  0.8713333333333333
std =  0.06680319353643706


### オンラインSVM

In [21]:
from sklearn import linear_model

X = bow
y = df['label']

clf = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)

scores=cross_val_score(clf, X, y, cv=10)
print("acc = ", scores.mean())
print("std = ", scores.std())

acc =  0.8876666666666667
std =  0.07096243137510626
